In [4]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import kinodyn_um.urdfparser as u2c
import numpy as np
import casadi as ca

In [5]:
# ─────────────────────────────────────────────────────────────────────────────
# 1. Load robot model
# ─────────────────────────────────────────────────────────────────────────────
alpha = u2c.URDFparser()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(project_root, 'usage', 'urdf', 'alpha_5_robot.urdf')
alpha.from_file(path_to_urdf)

root = "base_link"
tip  = "alpha_standard_jaws_base_link"
n_joints = alpha.get_n_joints(root, tip)

# ─────────────────────────────────────────────────────────────────────────────
# 2. CasADi symbols
# ─────────────────────────────────────────────────────────────────────────────
q     = ca.MX.sym('q',     n_joints)        # joint positions
q_dot = ca.MX.sym('q_dot', n_joints)        # joint velocities
tau   = ca.MX.sym('tau',   n_joints)        # joint torques
dt    = ca.MX.sym('dt')                     # integration step (s)
g     = ca.MX.sym('g')                      # gravity acceleration (e.g. -9.81)

k        = ca.MX.sym('k_',        n_joints, 2)   # tanh sharpness coefficients
viscous  = ca.MX.sym('viscous',   n_joints, 2)   # viscous friction
coulomb  = ca.MX.sym('coulomb',   n_joints, 2)   # Coulomb friction
I_Grotor = ca.MX.sym('I_Grotor',  n_joints, 2)   # rotor inertias
payload_W = ca.MX.sym('payload_W')              # payload weight (N)

lower_joint_limit = ca.MX.sym('lower_limit', n_joints)
upper_joint_limit = ca.MX.sym('upper_limit', n_joints)

# ─────────────────────────────────────────────────────────────────────────────
# 3. Forward dynamics (from CRBA model produced by urdf2casadi)
# ─────────────────────────────────────────────────────────────────────────────
qddot_sym = alpha.get_forward_dynamics_crba(root, tip)
q_ddot = qddot_sym(q, q_dot, tau, g, k, viscous, coulomb, I_Grotor, payload_W)

# ─────────────────────────────────────────────────────────────────────────────
# 4. Joint‑limit saturation with recovery
#    Freeze motion only if it is trying to go farther OUT of bounds.
# ─────────────────────────────────────────────────────────────────────────────
EPS = 1e-6  # small buffer to avoid numerical chatter

qdot_n  = ca.MX.zeros(n_joints)
qddot_n = ca.MX.zeros(n_joints)

for i in range(n_joints):
    # Are we near / beyond either limit?
    above = q[i] >= upper_joint_limit[i] - EPS
    below = q[i] <= lower_joint_limit[i] + EPS

    # Will current velocity / acceleration push farther out?
    vel_out_high = ca.logic_and(above,  q_dot[i]  > 0)
    vel_out_low  = ca.logic_and(below,  q_dot[i]  < 0)
    acc_out_high = ca.logic_and(above,  q_ddot[i] > 0)
    acc_out_low  = ca.logic_and(below,  q_ddot[i] < 0)

    block_vel = ca.logic_or(vel_out_high, vel_out_low)
    block_acc = ca.logic_or(acc_out_high, acc_out_low)

    qdot_n[i]  = ca.if_else(block_vel, 0, q_dot[i])
    qddot_n[i] = ca.if_else(block_acc, 0, q_ddot[i])

# ─────────────────────────────────────────────────────────────────────────────
# 5. State vector and continuous‑time ODE (scaled by dt for 0‑1 integrator)
# ─────────────────────────────────────────────────────────────────────────────
x     = ca.vertcat(q, q_dot)
xdot  = ca.vertcat(qdot_n, qddot_n) * dt   # integrate over [0,1] and scale by dt

# ─────────────────────────────────────────────────────────────────────────────
# 6. Parameter vector (keep one long vector to avoid argument mismatch)
# ─────────────────────────────────────────────────────────────────────────────
p = ca.vertcat(
        dt,
        g,
        ca.reshape(k,       -1, 1),
        ca.reshape(viscous, -1, 1),
        ca.reshape(coulomb, -1, 1),
        ca.reshape(I_Grotor,-1, 1),
        payload_W,
        lower_joint_limit,
        upper_joint_limit)
# ─────────────────────────────────────────────────────────────────────────────
# 7. Integrator (Runge–Kutta over [0,1])
# ─────────────────────────────────────────────────────────────────────────────
dae  = {'x': x, 'ode': xdot, 'p': p, 'u': tau}
opts = {
    'simplify': True,
    'number_of_finite_elements': 200,
}

intg = ca.integrator('intg', 'rk', dae, 0, 1, opts)

# ─────────────────────────────────────────────────────────────────────────────
# 8. Next‑state function and file export
# ─────────────────────────────────────────────────────────────────────────────
x_next = intg(x0=x, u=tau, p=p)['xf']

p_sim =ca.vertcat(
    ca.reshape(k,   -1, 1), 
    ca.reshape(viscous,   -1, 1),
    ca.reshape(coulomb,   -1, 1),
    ca.reshape(I_Grotor,  -1, 1)
        )

F_next = ca.Function('Mnext', [x, tau, dt, g, payload_W, p_sim , lower_joint_limit, upper_joint_limit], [x_next])

F_next.save('arm.casadi')

# c , cpp or matlab code generation for forward dynamics
F_next.generate("F_next_.c")
os.system(f"gcc -fPIC -shared F_next_.c -o libMnext.so")

print('✔ arm.casadi saved — ready for simulation')

✔ arm.casadi saved — ready for simulation
